In [11]:
import os
import json
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())
openai_api_key = os.environ["OPENAI_API_KEY"]
OPENROUTER_API_KEY = os.environ["OPENROUTER_API_KEY"]
os.environ["GROQ_API_KEY"]
os.environ["OPENROUTER_API_KEY"]
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_community.document_loaders.csv_loader import CSVLoader
from langchain_community.document_loaders import PyPDFLoader, TextLoader, Docx2txtLoader

In [52]:
from typing import Iterator
from agno.agent import Agent, RunResponse
from agno.models.openai import OpenAIChat
from agno.utils.pprint import pprint_run_response

agent = Agent(model=Groq(id="llama-3.3-70b-versatile"))

# Run agent and return the response as a variable
response: RunResponse = agent.run("Tell me a 5 second short story about a robot")
# Run agent and return the response as a stream
response_stream: Iterator[RunResponse] = agent.run("Tell me a 5 second short story about a lion", stream=True)

# Print the response in markdown format
pprint_run_response(response, markdown=True)

╭─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╮
│ As the clock struck midnight, a lone robot's eyes flickered to life, and it whispered, "I remember."            │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

In [87]:
from agno.agent import Agent
from agno.models.openai import OpenAIChat
from agno.team import Team
from agno.tools.duckduckgo import DuckDuckGoTools

# Create individual specialized agents
researcher = Agent(
    name="Researcher",
    role="Expert at finding information",
    tools=[DuckDuckGoTools()],
    model=OpenAIChat("gpt-4.1-nano"),
    #debug_mode=True
)

writer = Agent(
    name="Writer",
    role="Expert at writing clear, engaging content",
    model=OpenAIChat("gpt-4.1-nano"),
    #debug_mode=True
)

# Create a team with these agents
content_team = Team(
    name="Content Team",
    mode="coordinate",
    members=[researcher, writer],
    instructions="You are a team of researchers and writers that work together to create high-quality content.",
    model=OpenAIChat("gpt-4.1-nano"),
    markdown=True,
)

content_team.print_response("Create a short article about data  science in 2 lines after doing research")

Output()

In [88]:
response = content_team.run("Create a short article about data  science in 2 lines after doing research")
content_team.print_response(response.content)

Output()

In [89]:
from agno.agent import Agent, RunResponse
from agno.models.groq import Groq
from fpdf import FPDF

# Initialize agent
agent = Agent(
    model=Groq(id="llama-3.3-70b-versatile"),
    markdown=True
)

# Get response
response = agent.run("Tell me a joke")

# Print response
agent.print_response(response.content)


Output()

In [90]:
def load_file(file_path: str):
    """
    Load content from a PDF, Word, or text file.
    """
    if file_path.endswith(".pdf"):
        loader = PyPDFLoader(file_path)
    elif file_path.endswith(".txt"):
        loader = TextLoader(file_path)
    else:
        raise ValueError("Unsupported file format. Please provide a .pdf, .docx, or .txt file.")
    documents = loader.load()
    return documents#"\n".join([doc.page_content for doc in documents])


# Paths to the job description and resume files
jd = "JD.txt"  # Replace with your file path
cv = "CV.pdf"  # Replace with your file path

def load_cv_jd(jd1= None, cv1= None):

    # Load the files
    try:
        job_description = load_file(jd1)
        resume1 = load_file(cv1)
        
        rjd = "\n".join([doc.page_content for doc in job_description])
        rcv = "\n".join([doc.page_content for doc in resume1])
        return rjd, rcv
        
    except Exception as e:
        print(f"An error occurred: {e}")
    
JD, CV = load_cv_jd(jd,cv)
print(CV)

Raviteja R
Senior Product Data Analyst, 
Certified Data Scientist
r2raviteja@gmail.com
 
+49 15216 442399
 
Munich, Germany
 
rraviteja
 
Profile  with 5 years of experience in 
analytics, A/B testing and KPI optimization. 
Proficient in building scalable dashboards 
and machine learning models to drive 
business impact. Available for immediate 
joining.
Education
Masters in Business Analytics & Data 
Science, EU Business School
March 2023 – March 2025    
Munich, Germany
GPA : 1,6/4 (German Scale,  1 = Highest)
Post Graduate Diploma in Data Science, 
IIIT Bangalore
April 2019 – September 2020
GPA - 3.2/4 (U.S Scale, with 4 = Highest)
Bachelor of Technology (ECE) | 
Specialization in Data Science, 
Lovely Professional University
June 2015 – April 2019
GPA : 7.76/10 (with 10 = Highest)
Projects
Thesis: AI-Driven Interview Bot
Developed an AI-powered interview bot 
leveraging Generative AI, Langchain, RAG 
and NLP, reducing early-stage interviewer 
workload by 60% and increasing candidat

In [93]:
data_extractor_agent = Agent(
    name="Data Extractor",
    role="Extract and structure personal information from unstructured text",
    model=OpenAIChat("gpt-4.1-nano"),
    #tools=[ReasoningTools(add_instructions=True)],
    instructions=[
        "Extract specific fields from unstructured text input",
        "Identify and separate different data points",
        "Handle missing values (marked with '-') appropriately",
        "Return data in a structured dictionary format",
        "Verify extracted data for accuracy",
        "Handle both single-line and multi-line fields",
        "Preserve original formatting of URLs and special characters",
        "Convert empty fields to None or appropriate null representation"
    ],
    markdown=True,
)

In [97]:
query = """
    Extract the following fields from the resume below and return as JSON:
    - Email
    - password
    - First name
    - middle name (return None if '-')
    - Last name
    - Phone number
    - Language
    - Secondary Phone (return None if '-')
    - Street Address
    - City
    - State
    - Zip / Postal Code
    - Country
    - LinkedIn profile
    
    Example Input:
    Email abc@gmail.com
    default password = Asdfgh1234567@
    middle name -
    
    Example Output:
    {
        "Email": "abc@gmail.com",
        "password": "Asdfgh1234567@",
        "middle name": null
    }

    Resume to process:
    """+ CV
response = data_extractor_agent.run(query)
data_extractor_agent.print_response(response.content)

Output()

In [114]:
content_json = json.loads(response.content) 
content_json

{'Email': 'r2raviteja@gmail.com',
 'password': None,
 'First name': 'Raviteja',
 'middle name': None,
 'Last name': 'R',
 'Phone number': '+49 15216 442399',
 'Language': ['English', 'Telugu', 'German'],
 'Secondary Phone': None,
 'Street Address': None,
 'City': 'Munich',
 'State': 'Germany',
 'Zip / Postal Code': None,
 'Country': None,
 'LinkedIn profile': None}

In [113]:

content_json = json.loads(response.content) # {"key": "value"}  # Example content

# Specify the file path where you want to save the JSON
file_path = "user.txt"

# Write the JSON content to the text file
with open(file_path, 'w', encoding='utf-8') as file:
    json.dump(content_json, file, indent=4)  # indent for pretty formatting

print(f"JSON content has been written to {file_path}")

JSON content has been written to user.txt


In [106]:
import json
from agno.memory.v2.memory import Memory
from agno.memory.v2.db.sqlite import SqliteMemoryDb
from agno.memory.v2.schema import UserMemory

def get_memory_db():
    """Create and return a memory database instance"""
    return SqliteMemoryDb(
        table_name="user_data",
        db_file="storage/user_memory.db"
    )

def get_user_value(user_id, field_name):
    """
    Retrieve a specific field value for a user
    Args:
        user_id (str): The user's ID (email)
        field_name (str): The field name to retrieve (e.g., "First name", "Phone number")
    Returns:
        str: The value if found, None otherwise
    """
    memory_db = get_memory_db()
    memory = Memory(db=memory_db)
    
    # Get all memories for the user
    user_memories = memory.get_user_memories(user_id=user_id)
    
    # Search for the specific field
    for m in user_memories:
        if m.memory.startswith(f"{field_name}:"):
            # Extract the value part after the colon and strip whitespace
            return m.memory.split(":", 1)[1].strip()
    
    return None

def get_all_user_values(user_id):
    """
    Retrieve all stored values for a user as a dictionary
    Args:
        user_id (str): The user's ID (email)
    Returns:
        dict: Dictionary of field names and their values
    """
    memory_db = get_memory_db()
    memory = Memory(db=memory_db)
    
    user_values = {}
    user_memories = memory.get_user_memories(user_id=user_id)
    
    for m in user_memories:
        # Split memory string into field name and value
        if ":" in m.memory:
            field, value = m.memory.split(":", 1)
            user_values[field.strip()] = value.strip()
    
    return user_values

def store_user_data(user_content):
    """
    Store user data from JSON content into the memory database
    Args:
        user_content (str): JSON string containing user data
    """
    try:
        # Parse JSON if it's a string
        if isinstance(user_content, str):
            user_data = json.loads(user_content)
        else:
            user_data = user_content

        # Create a SQLite memory database
        memory_db = get_memory_db()

        # Initialize Memory with the storage backend
        memory = Memory(db=memory_db)

        # Use email as user_id if available, otherwise use a default
        user_id = user_data.get("Email", user_data.get("email", "default_user"))

        # Store each piece of user information as a separate memory
        for key, value in user_data.items():
            if value is not None:  # Only store non-null values
                # Clean the key for topic (remove special characters, lowercase)
                topic = key.lower().replace(" ", "_").replace(",", "").replace(".", "")
                
                memory.add_user_memory(
                    memory=UserMemory(
                        memory=f"{key}: {value}",
                        topics=["user_profile", topic]
                    ),
                    user_id=user_id
                )

        # Retrieve and display the stored memories
        print(f"\nStored User Memories for {user_id}:")
        user_memories = memory.get_user_memories(user_id=user_id)
        for m in user_memories:
            print(f"\nMemory: {m.memory}")
            print(f"Topics: {m.topics}")
            print(f"Last Updated: {m.last_updated}")

        return user_id  # Return user_id for further operations

    except json.JSONDecodeError as e:
        print(f"Error decoding JSON: {e}")
        return None
    except Exception as e:
        print(f"Error storing user data: {e}")
        return None

if __name__ == "__main__":
    # Example usage with user_content variable
    user_content = '''
    {
        "Email": "r2raviteja@gmail.com",
        "password": null,
        "First name": "Raviteja",
        "Phone number": "+49 15216 442399"
    }
    '''
    # Store the data
    user_id = store_user_data(content_json)
    
    if user_id:
        # Example: Get a specific value
        phone = get_user_value(user_id, "Phone number")
        first_name = get_user_value(user_id, "First name")
        print("\nRetrieved specific values:")
        print(f"Phone number: {phone}")
        print(f"First name: {first_name}")
        
        # Example: Get all values
        print("\nAll user values:")
        all_values = get_all_user_values(user_id)
        for field, value in all_values.items():
            print(f"{field}: {value}") 


Stored User Memories for r2raviteja@gmail.com:

Memory: Language: ['English', 'Telugu', 'German']
Topics: ['user_profile', 'language']
Last Updated: 2025-04-26 21:48:10.070535

Memory: City: Munich
Topics: ['user_profile', 'city']
Last Updated: 2025-04-26 21:48:10.427322

Memory: State: Germany
Topics: ['user_profile', 'state']
Last Updated: 2025-04-26 21:48:10.619538

Memory: Email: r2raviteja@gmail.com
Topics: ['user_profile', 'email']
Last Updated: 2025-04-26 21:48:09.196154

Memory: First name: Raviteja
Topics: ['user_profile', 'first_name']
Last Updated: 2025-04-26 21:48:09.583640

Memory: Last name: R
Topics: ['user_profile', 'last_name']
Last Updated: 2025-04-26 21:48:09.719617

Memory: Phone number: +49 15216 442399
Topics: ['user_profile', 'phone_number']
Last Updated: 2025-04-26 21:48:09.860890

Memory: City: Munich
Topics: ['user_profile', 'city']
Last Updated: 2025-04-26 21:44:04.121926

Memory: State: Germany
Topics: ['user_profile', 'state']
Last Updated: 2025-04-26 21:4

In [109]:
if user_id:
    # Example: Get a specific value
    #phone = get_user_value(user_id, "Phone number")
    #first_name = get_user_value(user_id, "First name")
    #print("\nRetrieved specific values:")
    #print(f"Phone number: {phone}")
    #print(f"First name: {first_name}")
    
    # Example: Get all values
    print("\nAll user values:")
    all_values = get_all_user_values(user_id)
    for field, value in all_values.items():
        print(f"{field}: {value}") 


All user values:
Language: ['English', 'Telugu', 'German']
City: Munich
State: Germany
Email: r2raviteja@gmail.com
First name: Raviteja
Last name: R
Phone number: +49 15216 442399


In [41]:
from agno.agent import Agent
from agno.models.openai import OpenAIChat
from agno.tools.reasoning import ReasoningTools

# Cover Letter Writer Agent
cover_letter_agent = Agent(
    name="Cover Letter Writer",
    role="Write professional and compelling cover letters",
    model=OpenAIChat(id="gpt-4.1-nano"),
    tools=[ReasoningTools(add_instructions=True)],
    instructions=[
        "Write professional and compelling cover letters",
        "Use formal business letter format",
        "Create engaging opening paragraphs",
        "Match qualifications with job requirements",
        "Include specific examples and achievements",
        "Maintain professional tone",
        "Keep content concise and impactful",
        "End with clear call to action"
    ],
    markdown=True
)

def generate_cover_letter(resume_text: str, job_description: str, applicant_info: dict, company_info: dict) -> str:
    """
    Generate a cover letter using the agent
    
    Args:
        resume_text: Full text of the resume/CV
        job_description: Full job description
        applicant_info: Dictionary with applicant's contact details
        company_info: Dictionary with company and position details
    
    Returns:
        str: The final cover letter
    """
    task = f"""
    Generate a professional cover letter with the following inputs:

    RESUME:
    {resume_text}

    JOB DESCRIPTION:
    {job_description}

    APPLICANT INFORMATION:
    {applicant_info}

    COMPANY INFORMATION:
    {company_info}
    """
    o1 =  cover_letter_agent.run(task)
    return o1

if __name__ == "__main__":
    # Example usage
    resume = """[Your resume text here]"""
    job_desc = """[Job description text here]"""
    applicant = {
        "name": "John Doe",
        "email": "john@example.com",
        "phone": "(555) 123-4567",
        "linkedin": "linkedin.com/in/johndoe"
    }
    company = {
        "name": "Tech Corp",
        "hiring_manager": "Jane Smith",
        "position": "Senior Software Engineer"
    }
    
    # Generate and print cover letter
    cover_letter = generate_cover_letter(CV, JD, applicant, company)
     

In [44]:
print(cover_letter.content)

[Your Name]  
[Your Address]  
Munich, Germany  
r2raviteja@gmail.com | +49 15216 442399  
  
[Date]  
  
Jane Smith  
Blacklane  
[Company Address]  
  
Dear Ms. Smith,  
  
I am writing to express my keen interest in the Senior Data Analyst position at Blacklane. With over five years of comprehensive experience in analytics, dashboard development, and driving strategic business insights, I am confident in my ability to contribute effectively to your data initiatives. My background in developing scalable dashboards, performing advanced statistical analyses, and collaborating across cross-functional teams aligns well with Blacklane’s mission to leverage data for impactful decision-making.  
  
Throughout my career, I have successfully designed and implemented data solutions that enhance operational efficiency and business growth. At Allianz Partners, I developed Power BI dashboards for executive-level insights across 119 business units, automating reports that reduced manual workload b

In [48]:
def save_to_word(content: str, filename: str = "cover_letter.docx"):
    """Save content to a Word document"""
    doc = Document()
    for paragraph in content.split('\n\n'):
        if paragraph.strip():
            doc.add_paragraph(paragraph.strip())
    doc.save(filename)
save_to_word(cover_letter.content)

In [47]:
with open("test.txt", 'w', encoding='utf-8') as f:
        f.write(cover_letter.content)

In [20]:
from agno.agent import Agent
from agno.models.openai import OpenAIChat
from agno.tools.reasoning import ReasoningTools

# Cover Letter Writer Agent
cover_letter_agent = Agent(
    name="Cover Letter Writer",
    role="Write professional and compelling cover letters",
    model=OpenAIChat(id="gpt-4.1-nano"),
    tools=[ReasoningTools(add_instructions=True)],
    instructions=[
        "Write professional and compelling cover letters",
        "Use formal business letter format",
        "Create engaging opening paragraphs",
        "Match qualifications with job requirements",
        "Include specific examples and achievements",
        "Maintain professional tone",
        "Keep content concise and impactful",
        "End with clear call to action"
    ],
    markdown=True
)

def generate_cover_letter(resume_text: str, job_description: str, applicant_info: dict, company_info: dict) -> str:
    """
    Generate a cover letter using the agent
    
    Args:
        resume_text: Full text of the resume/CV
        job_description: Full job description
        applicant_info: Dictionary with applicant's contact details
        company_info: Dictionary with company and position details
    
    Returns:
        str: The final cover letter
    """
    task = f"""
    Generate a professional cover letter with the following inputs:

    RESUME:
    {resume_text}

    JOB DESCRIPTION:
    {job_description}

    APPLICANT INFORMATION:
    {applicant_info}

    COMPANY INFORMATION:
    {company_info}
    """
    
    return cover_letter_agent.print_response(task, stream=True)

if __name__ == "__main__":
    # Example usage
    resume = """[Your resume text here]"""
    job_desc = """[Job description text here]"""
    applicant = {
        "name": "John Doe",
        "email": "john@example.com",
        "phone": "(555) 123-4567",
        "linkedin": "linkedin.com/in/johndoe"
    }
    company = {
        "name": "Tech Corp",
        "hiring_manager": "Jane Smith",
        "position": "Senior Software Engineer"
    }
    
    # Generate and print cover letter
    cover_letter = generate_cover_letter(CV, JD, applicant, company)
    print(cover_letter) 

WARNING  Could not run function think(...)

ERROR    1 validation error for ReasoningTools.think                                                               
         parameters                                                                                                
           Unexpected keyword argument [type=unexpected_keyword_argument, input_value={'tool_uses':                
         [{'recipien...", 'confidence': 0.9}}]}, input_type=dict]                                                  
             For further information visit https://errors.pydantic.dev/2.11/v/unexpected_keyword_argument          
         Traceback (most recent call last):                                                                        
           File "c:\Users\960ra\anaconda3\envs\ChatPdf\lib\site-packages\agno\tools\function.py", line 554, in     
         execute                                                                                                   
             result = self.function.entrypoint(**arguments)                                                        
           File "c:\Users\960ra\anaconda3\envs\ChatPdf\lib\site-packages\pydantic\_internal\_validate_call.py",    
         line 39, in wrapper_function                                                                              
             return wrapper(*args, **kwargs)                                                                       
           File "c:\Users\960ra\anaconda3\envs\ChatPdf\lib\site-packages\pydantic\_internal\_validate_call.py",    
         line 136, in __call__                                                                                     
             res = self.__pydantic_validator__.validate_python(pydantic_core.ArgsKwargs(args, kwargs))             
         pydantic_core._pydantic_core.ValidationError: 1 validation error for ReasoningTools.think                 
         parameters                                                                                                
           Unexpected keyword argument [type=unexpected_keyword_argument, input_value={'tool_uses':                
         [{'recipien...", 'confidence': 0.9}}]}, input_type=dict]                                                  
             For further information visit https://errors.pydantic.dev/2.11/v/unexpected_keyword_argument

WARNING  Could not run function think(...)

ERROR    1 validation error for ReasoningTools.think                                                               
         parameters                                                                                                
           Unexpected keyword argument [type=unexpected_keyword_argument, input_value={'recipient_name':           
         'funct...s.", 'confidence': 0.9}}, input_type=dict]                                                       
             For further information visit https://errors.pydantic.dev/2.11/v/unexpected_keyword_argument          
         Traceback (most recent call last):                                                                        
           File "c:\Users\960ra\anaconda3\envs\ChatPdf\lib\site-packages\agno\tools\function.py", line 554, in     
         execute                                                                                                   
             result = self.function.entrypoint(**arguments)                                                        
           File "c:\Users\960ra\anaconda3\envs\ChatPdf\lib\site-packages\pydantic\_internal\_validate_call.py",    
         line 39, in wrapper_function                                                                              
             return wrapper(*args, **kwargs)                                                                       
           File "c:\Users\960ra\anaconda3\envs\ChatPdf\lib\site-packages\pydantic\_internal\_validate_call.py",    
         line 136, in __call__                                                                                     
             res = self.__pydantic_validator__.validate_python(pydantic_core.ArgsKwargs(args, kwargs))             
         pydantic_core._pydantic_core.ValidationError: 1 validation error for ReasoningTools.think                 
         parameters                                                                                                
           Unexpected keyword argument [type=unexpected_keyword_argument, input_value={'recipient_name':           
         'funct...s.", 'confidence': 0.9}}, input_type=dict]                                                       
             For further information visit https://errors.pydantic.dev/2.11/v/unexpected_keyword_argument

WARNING  Could not run function think(...)

...

WARNING  Could not run function think(...)

ERROR    1 validation error for ReasoningTools.think                                                               
         parameters                                                                                                
           Unexpected keyword argument [type=unexpected_keyword_argument, input_value={'recipient_name':           
         'funct...s.", 'confidence': 0.9}}, input_type=dict]                                                       
             For further information visit https://errors.pydantic.dev/2.11/v/unexpected_keyword_argument          
         Traceback (most recent call last):                                                                        
           File "c:\Users\960ra\anaconda3\envs\ChatPdf\lib\site-packages\agno\tools\function.py", line 554, in     
         execute                                                                                                   
             result = self.function.entrypoint(**arguments)                                                        
           File "c:\Users\960ra\anaconda3\envs\ChatPdf\lib\site-packages\pydantic\_internal\_validate_call.py",    
         line 39, in wrapper_function                                                                              
             return wrapper(*args, **kwargs)                                                                       
           File "c:\Users\960ra\anaconda3\envs\ChatPdf\lib\site-packages\pydantic\_internal\_validate_call.py",    
         line 136, in __call__                                                                                     
             res = self.__pydantic_validator__.validate_python(pydantic_core.ArgsKwargs(args, kwargs))             
         pydantic_core._pydantic_core.ValidationError: 1 validation error for ReasoningTools.think                 
         parameters                                                                                                
           Unexpected keyword argument [type=unexpected_keyword_argument, input_value={'recipient_name':           
         'funct...s.", 'confidence': 0.9}}, input_type=dict]                                                       
             For further information visit https://errors.pydantic.dev/2.11/v/unexpected_keyword_argument

WARNING  Could not run function think(...)

ERROR    1 validation error for ReasoningTools.think                                                               
         parameters                                                                                                
           Unexpected keyword argument [type=unexpected_keyword_argument, input_value={'recipient_name':           
         'funct...s.", 'confidence': 0.9}}, input_type=dict]                                                       
             For further information visit https://errors.pydantic.dev/2.11/v/unexpected_keyword_argument          
         Traceback (most recent call last):                                                                        
           File "c:\Users\960ra\anaconda3\envs\ChatPdf\lib\site-packages\agno\tools\function.py", line 554, in     
         execute                                                                                                   
             result = self.function.entrypoint(**arguments)                                                        
           File "c:\Users\960ra\anaconda3\envs\ChatPdf\lib\site-packages\pydantic\_internal\_validate_call.py",    
         line 39, in wrapper_function                                                                              
             return wrapper(*args, **kwargs)                                                                       
           File "c:\Users\960ra\anaconda3\envs\ChatPdf\lib\site-packages\pydantic\_internal\_validate_call.py",    
         line 136, in __call__                                                                                     
             res = self.__pydantic_validator__.validate_python(pydantic_core.ArgsKwargs(args, kwargs))             
         pydantic_core._pydantic_core.ValidationError: 1 validation error for ReasoningTools.think                 
         parameters                                                                                                
           Unexpected keyword argument [type=unexpected_keyword_argument, input_value={'recipient_name':           
         'funct...s.", 'confidence': 0.9}}, input_type=dict]                                                       
             For further information visit https://errors.pydantic.dev/2.11/v/unexpected_keyword_argument

It appears there was an issue with how I formatted the tool call. I'll correct the structure and try again to generate the detailed sections of the cover letter.

Let's proceed with a properly formatted plan: I will generate the opening paragraph, body paragraphs emphasizing the candidate's relevant experience, and a closing paragraph. Afterward, I'll compile them into a coherent cover letter.

I will now make the correct tool call to mitigate the JSON error.


In [22]:
cover_letter_agent.print_response(response.content)

Output()

In [13]:
from agno.agent import Agent
from agno.models.openai import OpenAIChat
from agno.tools.reasoning import ReasoningTools

# Cover Letter Writer Agent
cover_letter_agent = Agent(
    name="Cover Letter Writer",
    role="Write professional and compelling cover letters",
    model=OpenAIChat(id="gpt-4.1-nano"),
    tools=[ReasoningTools(add_instructions=True)],
    instructions=[
        "Write professional and compelling cover letters",
        "Use formal business letter format",
        "Create engaging opening paragraphs",
        "Match qualifications with job requirements",
        "Include specific examples and achievements",
        "Maintain professional tone",
        "Keep content concise and impactful",
        "End with clear call to action"
    ],
    markdown=True
)

def generate_cover_letter(resume_text: str, job_description: str, applicant_info: dict, company_info: dict) -> str:
    """
    Generate a cover letter using the agent
    
    Args:
        resume_text: Full text of the resume/CV
        job_description: Full job description
        applicant_info: Dictionary with applicant's contact details
        company_info: Dictionary with company and position details
    
    Returns:
        str: The final cover letter
    """
    task = f"""
    Generate a professional cover letter with the following inputs:

    RESUME:
    {resume_text}

    JOB DESCRIPTION:
    {job_description}

    APPLICANT INFORMATION:
    {applicant_info}

    COMPANY INFORMATION:
    {company_info}
    """
    
    return cover_letter_agent.print_response(task, stream=True)

if __name__ == "__main__":
    # Example usage
    resume = """[Your resume text here]"""
    job_desc = """[Job description text here]"""
    applicant = {
        "name": "John Doe",
        "email": "john@example.com",
        "phone": "(555) 123-4567",
        "linkedin": "linkedin.com/in/johndoe"
    }
    company = {
        "name": "Tech Corp",
        "hiring_manager": "Jane Smith",
        "position": "Senior Software Engineer"
    }
    
    # Generate and print cover letter
    cover_letter = generate_cover_letter(CV, JD, applicant, company)
    print(cover_letter) 

Output()

None
